In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM
from tensorflow.keras.optimizers import Adam
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.envs import DummyVecEnv

np.random.seed(42)
tf.random.set_seed(42)

data = pd.DataFrame({
    'temperature': np.random.normal(22, 2, 1000),
    'humidity': np.random.normal(50, 10, 1000),
    'occupancy': np.random.randint(0, 2, 1000),
    'time_of_day': np.random.randint(0, 24, 1000),
    'energy_consumption': np.random.normal(50, 10, 1000)
})

data['time_of_day_sin'] = np.sin(2 * np.pi * data['time_of_day'] / 24)
data['time_of_day_cos'] = np.cos(2 * np.pi * data['time_of_day'] / 24)
data.drop(['time_of_day'], axis=1, inplace=True)

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

X = scaled_data[:, :-1]
y = scaled_data[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

model_lstm = tf.keras.Sequential([
    LSTM(64, activation='relu', input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]), return_sequences=False),
    Dense(1)
])

model_lstm.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
history = model_lstm.fit(X_train_lstm, y_train, epochs=50, batch_size=32, validation_data=(X_test_lstm, y_test))

predictions = model_lstm.predict(X_test_lstm)
plt.plot(y_test, label='True Energy Consumption')
plt.plot(predictions, label='Predicted Energy Consumption')
plt.legend()
plt.title('Energy Consumption Forecasting (LSTM)')
plt.show()

input_dim = X_train.shape[1]
input_layer = Input(shape=(input_dim,))
encoded = Dense(64, activation='relu')(input_layer)
encoded = Dense(32, activation='relu')(encoded)
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, validation_data=(X_test, X_test))

reconstructed = autoencoder.predict(X_test)
mse = np.mean(np.power(X_test - reconstructed, 2), axis=1)
threshold = np.percentile(mse, 95)
anomalies = mse > threshold
plt.scatter(np.arange(len(mse)), mse, label='MSE (Reconstruction Error)')
plt.axhline(threshold, color='r', linestyle='--', label='Anomaly Threshold')
plt.legend()
plt.title('Anomaly Detection with Autoencoders')
plt.show()

class EnergyEnv(gym.Env):
    def __init__(self):
        super(EnergyEnv, self).__init__()
        self.action_space = gym.spaces.Discrete(3)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=(4,))

    def step(self, action):
        temperature = np.random.normal(22, 2)
        humidity = np.random.normal(50, 10)
        occupancy = np.random.randint(0, 2)
        energy = temperature * 0.5 + humidity * 0.3 + occupancy * 0.2
        energy += action * 10
        reward = -energy
        done = False
        return np.array([temperature, humidity, occupancy, energy]), reward, done, {}

    def reset(self):
        return np.array([22, 50, 0, 50])

env = DummyVecEnv([lambda: EnergyEnv()])
model_rl = PPO("MlpPolicy", env, verbose=1)
model_rl.learn(total_timesteps=10000)

initial_state = np.array([22, 50, 0, 50])
state, reward, done, _ = env.step(1)

print("Initial Energy Consumption:", initial_state[3])
print("Optimized Energy Consumption:", state[3])

print("\nEnergy Efficiency Model Summary:")
print("1. Time-Series Forecasting: LSTM model trained for energy consumption prediction.")
print("2. Anomaly Detection: Autoencoder used to detect energy inefficiencies.")
print("3. Energy Optimization: Reinforcement learning model (PPO) for dynamic control.")